# Виконання Курсової Роботи
**Тема:** Дослідження механізму обробки виняткових ситуацій
**Мета:** Дослідити механізм обробки виняткових ситуацій та причини його застосування
на прикладі мов програмування C++ та C#.
**Завдання:** 
1. Вивчити причини застосування обробки виняткових ситуацій.
2. Дослідити механізм обробки виняткових ситуацій у мовах програмування C++ та C#.
3. Написати програми мовами C++ та C#, у яких ввести інформацію про виконавця та реалізувати завдання згідно з варіантом. При обчисленні арифметичного виразу та введенні інформації передбачити обробку виняткових ситуацій.

|Арифметичний вираз|Опис класу-виразу|Завдання|
|:----------------:|-----------------|--------| 
|![(lg(4xb-c)xa)/(b+c/d-1)](/src/formula.svg)|Змінні - операнди виразу; конструктори; методи встановлення значень об’єкта, обчислення виразу, отримання значення об’єкта|Створити масив об’єктів; обчислити вираз для кожного об’єкта, вивести результат|

## Теоретичне виконання
### Критичні точки програми 
При обчисленні даного арифметичного виразу з математичної точки зору ми маємо критичні точки у наступних місцях:
1. База логарифма менше або дорівнює нулеві
2. Ділення на нуль
    1. d - нуль *(c/d, d = 0)*
    2. Знаменник виразу - нуль *(b+c/d-1 = 0)*  
    
Враховуючи особливості математичних операцій заданих мов програмування маємо 2 підходи до виконання:
1. У C++ перевіряти значення у критичній точці до обчислення, та викликати виняткову ситуацію, якщо умова не виконана
2. У C# ловити виключення та викликати теж саме виключення з модифікаціями

Також оскільки значення обчислення буде зберігатися у класі, у змінній `result` зробимо його приватним, та створимо метод `getResult()`, який повертає значення виразу, чи викликає виняткову ситуацію, якщо воно ще не обчислене.


### Моделювання
#### Flow програми
Для перевірки дієздатності та коректного виконання програми на початковому етапі були вироблена наступна модель.

![chart](/src/First_flow.svg)

Інтерпретація одного і того самого класу буде виконана на трьох мовах програмування. [C++](C_PlusPlus.ipynb) та [C#](C_Sharp.ipynb) як основні, та [Python](Python.ipynb) як контрольна.
> Реалізація класів знаходится у посиланнях :point_up_2: 

З файлу певного формату (скоріш за все “\*.csv”) будемо отримувати групи з чотирьох чисел.  
Для кожної групи виконувати обчислення кожним з трьох класів, та записувати результат виконання у 3 окремі файли відповідно.
Після цього порівняємо результати та зробимо висновки, чи внесемо корективи.

In [1]:
from random import randint, uniform, choice


class Generator:
    def __init__(self,file="start_values.tsv", number=500, bottom=-1000, top=1000, error_chance=13):
        self.file = file
        self.number = number
        self.floor = bottom
        self.top = top
        self.error_chance = error_chance
        
    def generate(self):
        with open(self.file, 'w') as table:
            for _ in range(self.number):
                
                values = {letter:round(uniform(self.floor, self.top), 4)
                          for letter in ('a', 'b', 'c', 'd')}
                
                error = randint(0,100)
                if error <= self.error_chance:
                    type = choice(['l', 'z1', 'z2'])
                    if type == 'l':
                        values['c'] = 4*values['b']
                    elif type == 'z1':
                        values['d'] = 0
                    else:
                        rand_b = randint(-10, 10)
                        values['d'] = randint(-100, 100)
                        values['c'] = values['d'] * rand_b
                        values['b'] = -(rand_b - 1)
                if _ == (self.number - 1):
                    ending = ''
                else:
                    ending = '\r\n'
                line = '\t'.join(str(v) for v in values.values()) + ending
                table.writelines(line)
            

## Підготовка
Після декількох спроб був написаний генератор з наступними можливостями.
Генерується n груп випадкових чисел з плаваючою комою з точністю до 4 цифр після коми, по 4 числа в кожній групі.
> У фінальній реалізації генерується 100 груп, для зменшення абсолютної кількості warnings

Числа генеруються у межах від -400 до 1000.*Діапазон від’ємних даних менше, ніж діапазон додатних задля зменшення кількості некоректних баз логарифму*  
З шансом 13 відсотків будуть спеціально згенеровані числа, що викликають 1 з 3 помилок в обчисленнях.  
Числа записуються у файл формату `tsv`. Цей формат був обраний через те, що для зчитування даних для C++ він значно зручніший за `csv`, а для C# та для Python це не має значення.

In [2]:
from IPython.display import display
from python_utils.auto_widgets import initialize
widg = initialize()
f_name = widg[0]
number = widg[1]
bottom = widg[2]
top = widg[3]
error = widg[4]
display(f_name, number, bottom, top, error)

Text(value='start_values.tsv', description="Ім'я файлу:")

IntSlider(value=100, continuous_update=False, description='Кількість:', layout=Layout(width='65%'), max=1000, …

IntSlider(value=-400, continuous_update=False, description='Від:', layout=Layout(width='65%'), max=-1, min=-10…

IntSlider(value=1000, continuous_update=False, description='До:', layout=Layout(width='65%'), max=1000, min=10…

IntSlider(value=13, continuous_update=False, description='Шанс помилки:', layout=Layout(width='65%'), max=25)

In [3]:
generator = Generator(file=f_name.value, number=number.value, bottom=bottom.value, top=top.value, error_chance=error.value)
generator.generate()

### Структура класу
У ході виконання був розроблений клас `Handler`, який зберігає у собі 1-не приватне значення `result` - результат виразу, 4 публічних значення `a`, `b`, `c`, `d` - операнди виразу, та 3 публічні методи `getResult`, `calculate`, `logger`.

![chart](/src/Handler.svg)

Метод `getResult` повертає значення, що зберігається в `result`, або викликає виняткову ситуацію *NotImplemented*, якщо воно ще не обчислено.

Метод `calculate` виконує обчислення використовуючи операнди, що зберігаються у класі.
* **Приймає** булєвий параметр, який визначає чи виводити кожну окрему дію у консоль.
* **Записує** результат обчислення у змінну result.
* **Повертає** масив зі значенням кожної окремої дії (крім останньої).
* **Викликає** *Value(InvalidData)* або *ZeroDivision* виняткові ситуації за некоректної бази логарифму, або при діленні на нуль відповідно.  

Метод `logger` працює із лог файлом. Він виконує обчислення виразу та записує результат та усі проміжні обчислення у цей файл. Якщо йому трапляється виняткова ситуація, - то її код буде записаний у той самий файл замість результатів.

## Виконання
Після генерації файлу зі значеннями викликаємо виконання обчислень на кожній з 3 мов у background'і.

При виконанні код буде брати групу з 4 значень з файлу, виконувати обчислення з нею та записувати результат, або код помилки у файл “назва_мови_програмування.log.csv” та переходити до наступної групи.

|Код |Виняток|Опис|
|---:|-------|----|
|e0|ZeroDivisionError|Виникає при діленні на нуль|
|eL|ValueError / InvalidDataError|Виникає при некоректній базі логарифма|
|eI|NotImplementedError|Виникає за відсутності даних про результат у класі|
|e?|Інша|Будь яке інше виключення|

Тепер файл зберігається у форматі csv, бо порівняння результатів буде виконуватися мовою Python, а усі маніпуляції з даними за допомогою бібліотеки pandas, яка з коробки має функціонал для обробки csv файлів.

In [4]:
from python_utils.kernel_runner import runner

runner()

Відкриття файлів
Налаштування
Виконується Python
Виконується C#
Виконується C++
Запис до файлів
Успішно завершено!


## Обробка результатів
Останнім етапом є обробка даних отриманих з трьох різних джерел.
Для цього був розроблений наступний алгоритм.  
Проітеруємо отримані дані рядками. Порівнюємо їх попарно, кожне з кожним, (*С#* vs *Python*; *C++* vs *Python*; *C#* vs *C++*) та зберігаємо відповідний результат. Після цього перевіримо чи однакові всі три значення.

In [5]:
def compare(*args):
    length = min(len(i) for i in args)
    result = [] 
    for index in range(length):
        checker = [i[index] for i in args]
        try:
            result.append(all(round(float(x), 4) == round(float(checker[0]), 4)
                              for x in checker))
        except ValueError:
            result.append(all(x == checker[0] for x in checker))
    return result

In [6]:
import pandas as pd

py_data = pd.read_csv("logs/python.log.csv", header=0).round(5)
cs_data = pd.read_csv("logs/c#.log.csv", header=0).round(5)
cpp_data = pd.read_csv("logs/c++.log.csv", header=0).round(5)

In [7]:
conclusion = pd.DataFrame({
    "Py_Cs": compare(py_data.Result, cs_data.Result),
    "Py_Cpp": compare(py_data.Result, cpp_data.Result),
    "Cs_Cpp": compare(cs_data.Result, cpp_data.Result),
    "Py_Cs_Cpp": compare(py_data.Result, cs_data.Result, cpp_data.Result)
})

Якщо усі три значення рівні, додаємо їх до `Success`, якщо ні - до `Failed`. Після цього проітеруємо усі значення `Failed`. Якщо з трьох пар хоча б у одній однакові значення,  результат видаляється з `Failed` та записується у `Warnings`.

In [8]:
warnings = []

for index in range(len(conclusion.Py_Cs_Cpp)):
    if not conclusion.Py_Cs_Cpp[index]:
        if any((conclusion.Py_Cs[index],
                conclusion.Py_Cpp[index],
                conclusion.Cs_Cpp[index])):
            warnings.append(index)

Після цього виводимо результати на екран.

In [13]:
from IPython.display import display, HTML, Math
from collections import Counter
from python_utils.warnings_display import warnings_list

quantity = Counter(conclusion.Py_Cs_Cpp)
total = number.value
results = f"""
<h3>Результат</h3>
<table>
    <tr>
        <th>Тип</th>
        <th>Success</th>
        <th>Failed</th>
        <th>Warnings</th>
    </tr>
    <tr>
        <td>Кількість</td>
        <td>{quantity[True]} / {total}</td>
        <td>{quantity[False] - len(warnings)} / {total}</td>
        <td>{len(warnings)} / {total}</td>
    </tr>
    <tr>
        <td>Процент</td>
        <td>{(quantity[True] / total)*100}%</td>
        <td>{((quantity[False] - len(warnings)) / total)*100}%</td>
        <td>{(len(warnings) / total)*100}%</td>
    </tr>
</table>
"""
display(HTML(results))
if len(warnings) > 0:
    data = warnings_list(warnings, conclusion, py_data, cs_data, cpp_data)
    display(HTML("<h3>Warnings</h3>"))
    for warn in data:
        display(HTML(warn[0]))
        display(Math(warn[1]))

Тип,Success,Failed,Warnings
Кількість,99 / 100,0 / 100,1 / 100
Процент,99.0%,0.0%,1.0%


Index,Py vs C#,Py vs C++,C# vs C++
86,✔,✖,✖


<IPython.core.display.Math object>

![chart](/src/Second_flow.svg)

## Висновки
* В ході роботи були створені та протестовані програми з однаковою функціональністю написані на різних мовах програмування. Вони виконували обчислення за схемою, та за необхідністю обробляли виняткові ситуації.
* Після перевірки ми бачимо, що зі 100 випадково згенерованих значень ми маємо лише 1 попередження, 0 помилок.
* При детальному дослідженні випадків 'Warnings' було помічено, що у деяких випадках C++ множить числа з плаваючою комою з меншою точністю, ніж Python або C#.
* На жаль не вдалося встановити закономірність виникнення такої розбіжності.
* Оскільки 'Warnings' виникали лише у одному випадку в обчисленнях, й враховуючи їх результат залишається приблизно однаковим у всіх трьох випадках можна сказати, що поставлена мета була досягнута.
* Була написана програма, що коректно обчислює вирази за схемою ![(lg(4xb-c)xa)/(b+c/d-1)](/src/formula.svg) , та за необхідності викликає виняткові ситуації, повідомляючи, що не може завершити обчислення.
